In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.driver.port", "5001") \
        .getOrCreate()
spark.stop() # just to check whether port is available.  might need to restart your IDE.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 13:16:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:


from pyspark.sql import SparkSession
import pyspark.sql.dataframe
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# options
option_long = False
option_calc_correlation = False


spark = SparkSession.builder\
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.driver.port", "5001") \
        .config('spark.ui.port', '4050')\
        .getOrCreate()
        

sc = spark.sparkContext
#sc.setLogLevel("INFO")
sc.setLogLevel("WARN")

trainFileName = './train.csv'
testFileName = './test.csv'

# deerfootRDD = (sc.textFile(fileName, 8))
# print('\n'.join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0])).take(15)))

# Load a CSV file
trainDF = spark.read.csv(trainFileName, header=True, inferSchema=True)
testDF = spark.read.csv(testFileName, header=True, inferSchema=True)

print(trainDF.head())
print(testDF.head())

def corr(data: pyspark.sql.dataframe.DataFrame):
    plt.figure(figsize=(10, 10))
    sns.set_theme(style='whitegrid', context='notebook')


    # Get only numeric columns
    cols = [col for col, dtype in data.dtypes if dtype in ('int', 'double', 'float64')]

    # Initialize an empty dictionary to store correlations
    corr_dict = {}
    # Compute correlation matrix for numeric columns
    print("Calculating approx " + str(len(cols) * len(cols) / 2.0) + "correlations")
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):  # Ensure each pair is computed only once
            col1, col2 = cols[i], cols[j]
            corr_value = data.stat.corr(col1, col2)
            corr_dict[(col1, col2)] = corr_value

    corr_matrix = pd.DataFrame(index=cols, columns=cols)

    # Populate the correlation matrix
    for (col1, col2), corr_value in corr_dict.items():
        corr_matrix.at[col1, col2] = corr_value
        corr_matrix.at[col2, col1] = corr_value

    corr_matrix = corr_matrix.astype(float)
    sns.heatmap(corr_matrix , annot=True, square=False, cmap='coolwarm')
    plt.show()


if option_calc_correlation:
    corr(trainDF)


Row(MONTH=7, DAY_OF_WEEK=7, DEP_DEL15=0, DEP_TIME_BLK='1500-1559', DISTANCE_GROUP=3, SEGMENT_NUMBER=3, CONCURRENT_FLIGHTS=26, NUMBER_OF_SEATS=160, CARRIER_NAME='American Airlines Inc.', AIRPORT_FLIGHTS_MONTH=19534, AIRLINE_FLIGHTS_MONTH=79247, AIRLINE_AIRPORT_FLIGHTS_MONTH=7972, AVG_MONTHLY_PASS_AIRPORT=2006675, AVG_MONTHLY_PASS_AIRLINE=11744595, FLT_ATTENDANTS_PER_PASS=9.82082928995461e-05, GROUND_SERV_PER_PASS=0.00017728721959309724, PLANE_AGE=6, DEPARTING_AIRPORT='Douglas Municipal', LATITUDE=35.219, LONGITUDE=-80.936, PREVIOUS_AIRPORT="Chicago O'Hare International", PRCP=0.0, SNOW=0.0, SNWD=0.0, TMAX=95.0, AWND=4.25, CARRIER_HISTORICAL=0.2377091471954006, DEP_AIRPORT_HIST=0.2732357375614027, DAY_HISTORICAL=0.22253848170480506, DEP_BLOCK_HIST=0.25547875927174646)
Row(MONTH=4, DAY_OF_WEEK=6, DEP_DEL15=0, DEP_TIME_BLK='1000-1059', DISTANCE_GROUP=3, SEGMENT_NUMBER=3, CONCURRENT_FLIGHTS=5, NUMBER_OF_SEATS=110, CARRIER_NAME='Delta Air Lines Inc.', AIRPORT_FLIGHTS_MONTH=6837, AIRLINE_FLIG

24/12/03 13:16:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:

from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Prepare data
target_column = "DEP_DEL15"
label_column = "label"
numeric_features_name = "numeric_features"
numeric_features_name_scaled = "numeric_features_scaled"
features_name = "features"
features_name_scaled = "features_scaled"

numeric_features = [
    #  "MONTH",
    # "DAY_OF_WEEK",
    # "DISTANCE_GROUP",
    "SEGMENT_NUMBER",
    # "CONCURRENT_FLIGHTS",
    # "NUMBER_OF_SEATS",
    # "AIRPORT_FLIGHTS_MONTH",
    # "AIRLINE_AIRPORT_FLIGHTS_MONTH",
    # "AVG_MONTHLY_PASS_AIRPORT",
    # "AVG_MONTHLY_PASS_AIRLINE",
    # "FLT_ATTENDANTS_PER_PASS",
    # "GROUND_SERV_PER_PASS",
    # "PLANE_AGE",
    # "LATITUDE",
    # "LONGITUDE",
    "PRCP",
    # "SNOW",
    #"SNWD",
    #"TMAX",
    #"AWND",
    "CARRIER_HISTORICAL",
    "DEP_AIRPORT_HIST",
    "DAY_HISTORICAL",
    "DEP_BLOCK_HIST"
]

string_features = [
#    'DEP_TIME_BLK',
    'CARRIER_NAME',
    'DEPARTING_AIRPORT',
#    'PREVIOUS_AIRPORT'
]

def one_hot_name(for_string_feature):
    return for_string_feature + "_one_hot"
def indexer_name(for_string_feature):
    return for_string_feature + "_index"
def scaled_string(forString):
    return forString + "_scaled"

numeric_features_scaled = list(map(lambda feature: scaled_string(feature), numeric_features))
numeric_assembler = VectorAssembler(
    inputCols= numeric_features,
    outputCol=numeric_features_name
)

scaler = StandardScaler(inputCol=numeric_features_name, outputCol=numeric_features_name_scaled , withMean=True, withStd=True)

string_feature_indexers = list(map(lambda string_feature: StringIndexer(inputCol=string_feature, outputCol=indexer_name(string_feature), handleInvalid="keep"), string_features))
one_hot_encoders = list(map(lambda string_feature: OneHotEncoder(inputCol=indexer_name(string_feature), outputCol=one_hot_name(string_feature)), string_features))
label_indexer = StringIndexer(inputCol=target_column, outputCol=label_column)
one_hot_column_names = list(map(lambda string_feature: one_hot_name(string_feature), string_features))
assembler = VectorAssembler(
    inputCols= one_hot_column_names + numeric_features,
    outputCol=features_name
)

# classifier = LogisticRegression(
#     family='binomial',
#     fitIntercept=True,
#     featuresCol=features_name,
#     # regParam=0.1,  # Regularization parameter
#     # elasticNetParam=0.8,  # ElasticNet mixing parameter
#     maxIter=100,  # Number of iterations
#     threshold=0.5  # Adjusted threshold for rare class detection
# )

pipeline_preprocessing_stages =  string_feature_indexers + one_hot_encoders + [numeric_assembler, scaler, label_indexer, assembler]
preprocessing_pipeline = Pipeline(stages=pipeline_preprocessing_stages)
preprocessing_transformed = preprocessing_pipeline.fit(trainDF).transform(trainDF) # to calculate input dimension of MLP
input_dim = len(preprocessing_transformed.select(features_name).first()[features_name])
print(f"Input Dimension: {input_dim}")
binomial_classification_num_classes = 2
mlp_num_hidden_layers = 2 if option_long else 1
mlp_layer_node_size = 25 if option_long else 10
mlp_hidden_layers_list = [mlp_layer_node_size] * mlp_num_hidden_layers
classifier = MultilayerPerceptronClassifier(layers = [input_dim] + mlp_hidden_layers_list + [binomial_classification_num_classes])
pipeline_stages = pipeline_preprocessing_stages + [classifier]
pipeline = Pipeline(stages=pipeline_stages)
model = pipeline.fit(trainDF)

train_predictions = model.transform(trainDF)
test_predictions = model.transform(testDF)


Input Dimension: 119


24/12/03 13:16:53 WARN MemoryStore: Not enough space to cache rdd_106_2 in memory! (computed 49.5 MiB so far)
24/12/03 13:16:53 WARN MemoryStore: Not enough space to cache rdd_106_7 in memory! (computed 49.5 MiB so far)
24/12/03 13:16:53 WARN MemoryStore: Not enough space to cache rdd_106_3 in memory! (computed 49.5 MiB so far)
24/12/03 13:16:53 WARN MemoryStore: Not enough space to cache rdd_106_1 in memory! (computed 49.5 MiB so far)
24/12/03 13:16:53 WARN MemoryStore: Not enough space to cache rdd_106_0 in memory! (computed 49.5 MiB so far)
24/12/03 13:16:53 WARN BlockManager: Persisting block rdd_106_1 to disk instead.
24/12/03 13:16:53 WARN BlockManager: Persisting block rdd_106_0 to disk instead.
24/12/03 13:16:53 WARN BlockManager: Persisting block rdd_106_2 to disk instead.
24/12/03 13:16:53 WARN BlockManager: Persisting block rdd_106_3 to disk instead.
24/12/03 13:16:53 WARN BlockManager: Persisting block rdd_106_7 to disk instead.
24/12/03 13:16:53 WARN MemoryStore: Not enoug

In [4]:
fitted_classifier = model.stages[len(pipeline_stages) - 1]
feature_names = assembler.getInputCols()

# Print coefficients alongside feature names
# print("Feature Name -> Coefficient")
# for feature, coefficient in zip(feature_names, fitted_classifier.coefficients):
#     print(f"{feature} -> {coefficient}")

# print("Intercept:", fitted_classifier.intercept)
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol=label_column,
    predictionCol="prediction", 
    metricName="accuracy"  # accuracy for simplicity
)

evaluator_pr = BinaryClassificationEvaluator(
    labelCol=label_column, 
    rawPredictionCol="rawPrediction", 
    metricName="areaUnderPR"  # Use Area under PR curve for rare cases
)

# Define the positive class
positive_class = 1

from pyspark.sql.functions import col

# Calculate True Positives (TP) and False Negatives (FN)
tp_train = train_predictions.filter((col("label") == positive_class) & (col("prediction") == positive_class))
fn_train = train_predictions.filter((col("label") == positive_class) & (col("prediction") != positive_class))
tp_test = test_predictions.filter((col("label") == positive_class) & (col("prediction") == positive_class))
fn_test = test_predictions.filter((col("label") == positive_class) & (col("prediction") != positive_class))

# Calculate recall for the positive class
recall_train = tp_train.count() / (tp_train.count() + fn_train.count())
recall_test = tp_test.count() / (tp_test.count() + fn_test.count())

# Validation Accuracy
train_accuracy = evaluator.evaluate(train_predictions)
train_area_under_pr = evaluator_pr.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy:.6f}")
print(f"Training Recall: {recall_train:.6f}")
print(f"Training Area Under PR: {train_area_under_pr:.6f}")


test_accuracy = evaluator.evaluate(test_predictions)
test_area_under_pr = evaluator_pr.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy:.6f}")
print(f"Test Recall: {recall_test:.6f}")
print(f"Test Area Under PR: {test_area_under_pr:.6f}")


Training Accuracy: 0.811850
Training Recall: 0.019332
Training Area Under PR: 0.334703


Test Accuracy: 0.811859
Test Recall: 0.019603
Test Area Under PR: 0.334699


In [5]:
print("### False Negatives: " + str(fn_test.count()))
print(fn_test.show(5, truncate=False))
print("### True Positives" + str(tp_test.count()))
print(tp_test.show(5, truncate=False))

### False Negatives: 360992
+-----+-----------+---------+------------+--------------+--------------+------------------+---------------+----------------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+---------------------+---------+-----------------------------+--------+---------+--------------------------+----+----+----+----+-----+-------------------+-------------------+-------------------+-------------------+------------------+-----------------------+--------------------+-------------------------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+-----

### True Positives7218
+-----+-----------+---------+------------+--------------+--------------+------------------+---------------+----------------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+---------------------+---------+-----------------------------------------+--------+---------+----------------------------+----+----+----+----+-----+-------------------+-------------------+-------------------+-------------------+------------------+-----------------------+--------------------+-------------------------+------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------+------------------------------